In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('census_data_trimmed.csv')
df.head()

In [ ]:
updates = pd.read_csv('wtp_phonebook_update.csv')
updates.head()

In [ ]:
whitelist = pd.read_csv('whitelist_pretty.csv').drop(columns=['Unnamed: 0'])
whitelist.head()

In [ ]:
set(updates.columns) & set(df.columns)

In [ ]:
set(updates.columns) - set(df.columns)

In [ ]:
set(df.columns) - set(updates.columns)

In [ ]:
update_cols = ['village', 'ownername', 'subsector_swa']
df_cols = ['kijiji', 'ownername', 'subsector_swa_1']

In [ ]:
updates[update_cols]

In [ ]:
updates[update_cols].duplicated().sum()

In [ ]:
df[df_cols]

In [ ]:
df[df[df_cols].duplicated() | df[df_cols].duplicated(keep='last')][['firm_name'] + df_cols].sort_values(by=df_cols)

In [ ]:
update_ids = updates[update_cols[0]].apply(lambda x: x.strip().lower())
for col in update_cols[1:]:
    update_ids += updates[col].apply(lambda x: x.strip().lower())
    
print(update_ids.duplicated().sum())
print(update_ids)

In [ ]:
df_ids = df[df_cols[0]].apply(lambda x: x.strip().lower() if pd.notna(x) else x)
for col in df_cols[1:]:
    df_ids += df[col].apply(lambda x: x.strip().lower() if pd.notna(x) else x)

print(df_ids.duplicated().sum())
print(df_ids)

In [ ]:
set(update_ids) & set(df_ids)

In [ ]:
df[df_ids.isin(update_ids)].duplicated().sum()

In [ ]:
new_firms = updates[updates['census_firm'] == 0]
new_firms

In [ ]:
modified_firms = updates[updates['census_firm'] == 1]
modified_firms

In [ ]:
def id_from_cols(df, cols):
    ids = df[cols[0]].apply(lambda x: x.strip().lower() if pd.notna(x) else x)
    for col in cols[1:]:
        ids += df[col].apply(lambda x: x.strip().lower() if pd.notna(x) else x)
    return ids
    # return df[cols].apply(lambda x: hash(tuple(x)), axis=1)

In [ ]:
df['merge_id'] = id_from_cols(df, df_cols)

In [ ]:
modified_firms['merge_id'] = id_from_cols(modified_firms, update_cols)

In [ ]:
modified_firms['firm_name_noncensus'].value_counts()

In [ ]:
modified_firms['new_number'].value_counts()

In [ ]:
set(modified_firms['merge_id'].values) - set(df['merge_id'].values)

In [ ]:
modified_firms[modified_firms['merge_id'] == 'bugombeabera kabale titoduka la bidhaa tofauti'][['ownername', 'firm_name', 'village', 'subsector_swa', 'subsector_swa_eng', 'census_firm']]

In [ ]:
df[(df['kijiji'] == 'Bugombe') & (df['ownername'] == 'Abera Kabale Tito')][['firm_name', 'mobile_number1', 'mobile_number2', 'subsector_swa_1', 'kijiji', 'ownername']]

In [ ]:
modified_firms = modified_firms.drop(columns=['census_firm', 'firm_id_census', 'mobile_census1', 'mobile_census2','subsector_swa_eng','firm_name_noncensus','ownername'])
modified_firms = modified_firms.rename(columns={'mobile_final1': 'mobile_number1', 'mobile_final2': 'mobile_number2', 'subsector_swa': 'subsector_swa_1', 'subector_eng': 'subsector_eng_1', 'village': 'kijiji', 'ownername_new': 'ownername'})

modified_firms['sim_1'] = modified_firms['sim_2'] = "Vodacom"
modified_firms.loc[modified_firms['mobile_number2'].isna(), 'sim_2'] = pd.NA

In [ ]:
for id, row in modified_firms.iterrows():
    merge_id = row['merge_id']
    is_new_number = row['new_number']
    row = row.drop('merge_id').drop('new_number')
    
    if not is_new_number:
        row.drop('mobile_number1').drop('mobile_number2').drop('sim_1').drop('sim_2')

    assert merge_id in df['merge_id'].values

    # drop na values from the row
    row = row.dropna()
    
    # overwrite the row in df with the new data
    df.loc[df['merge_id'] == merge_id, row.index] = row